# ID3决策树

最近在看《机器学习实战》这本书，主要是为了给自己的机器学习中没有学过的知识补充一下，比如这个决策树我就没学过，但是这本书写的吧……还行，有些地方说得不是很清楚，正好学完了决策树也想巩固一下，于是写下此文。

其实我觉得决策树很像一种专家系统，有各种条件判断的分支，来判断符不符合标准、属于哪个类别……我在[某人的博客](http://www.cnblogs.com/bourneli/archive/2013/03/15/2961568.html)上找到了一个非常典型的图，银行可以根据这张图来判断是否可以放贷给这个人（当然，实际肯定比这幅图复杂得多）：  
![放贷决策树](./放贷决策树.png)
好了，这就是决策树了，我们根据各个分支，层层筛选，最后判断这个人是否有偿还贷款的能力，没有的话，就不贷款给他了。现在我给你一组数据（书上的数据），我要求，根据不浮出水面是否能生存、有没有脚蹼这两个特征来判断是否属于鱼类，用决策树来表现出来。  


|id|不浮出水面是否能生存|有没有脚蹼| 鱼类|
|:----:| :-------------: |:-------------:|:-----:|
|0| 是   | 是 | 是 |
|1| 是   | 是 | 是 |
|2| 是   | 否 | 否 |
|3| 否   | 是 | 否 |
|4| 否   | 是 | 否 |  
这个时候可能大家就会有疑惑了，到底第一个特征是用能否浮出水面生存来划分好呢，还是用有没有脚蹼来划分好？有什么评价标准吗？答案是用信息增益。

## 信息熵与信息增益
先来信息熵的公式亮瞎眼：  
$H=-\sum_{i=1}^{n}p\left ( x_{i} \right )log_{2}\left ( x_{i} \right )$  
伟大的香农提出来的（本人学通信的），n代表的是分类的数目，比如上面的表格中，只有两种类别，是鱼类，不是鱼类，我们用1表示是，0表示否，则不是鱼类的概率为：$p\left ( x_{0} \right )=\frac{3}{5}$，是鱼类的概率为：$p\left ( x_{1} \right )=\frac{2}{5}$我们手算一下目前的信息熵：  
$H=-\frac{3}{5}log_{2}\left ( \frac{3}{5} \right )-\frac{2}{5}log_{2}\left ( \frac{2}{5} \right )\approx 0.971$  
那么信息熵和我们设计决策树有什么关系呢？我们每一次选择某一特征作为节点进行划分的时候，都有一个目标——使得**这一节点**（注意，不是整体）的熵下降最多，也就是信息增益最大。  
假设我们以能浮出水面生存作为第一个节点来划分，那么1,2,3作为左子树，4,5作为右子树。在左子树中，是鱼类占$\frac{2}{3}$，不是鱼类占$\frac{1}{3}$，所以左子树的信息熵$H=-\frac{2}{3}log_{2}\left ( \frac{2}{3} \right )-\frac{1}{3}log_{2}\left ( \frac{1}{3} \right )\approx 0.918$；在右子树中，是鱼类占0%，不是鱼类占100%，所以右子树的信息熵为：$H=-1log_{2}\left ( 1 \right )=0$；左子树有1,2,3三个条目，占当前节点的$\frac{3}{5}$，右子树有4,5两个条目，占当前节点的$\frac{2}{5}$，所以按照能否浮出水面生存来划分的信息熵为$H=\frac{3}{5}*0.918+\frac{2}{5}*0 \approx 0.551$相对于未划分前，信息增益为0.420。  
那如果以有没有脚蹼作为划分标准，则1,2,4,5作为左子树，3作为右子树。在左子树中，是鱼类和不是鱼类各占$\frac{1}{2}$，左子树的信息熵$H=-2*\frac{1}{2}log_{2}\left ( \frac{1}{2} \right )=1$；右子树不是鱼类为100%，则右子树的信息熵为$H=-1log_{2}\left ( 1 \right )=0$；左子树有4个条目，占当前节点的$\frac{4}{5}$，右子树有1个条目，占当前节点的$\frac{1}{5}$，所以按照能否浮出水面生存来划分的信息熵为$H=\frac{4}{5}*1+\frac{1}{5}*0 \approx 0.8$相对于未划分前，信息增益为0.171。  
因此，我们选择信息增益最大的划分方案，划分结果如下图：  
![是否是鱼分类1](./是否是鱼分类1.png)

## 递归和停止条件
如果学过数据结构与算法的人应该知道，通常我们画树都是用递归来画的，那么递归截止的条件是啥？可以分两种情况：1、所有的分类都一样了，比如我们上图的右子树，都是鱼类，所有这个右子树可以停止继续递归了；2所有的特征都用完了，假如我们继续对左子树分类，把脚蹼作为分类特征，分完后已经没有其他特征可以继续分类了，这时候也不得不结束递归，但是又有一个问题了，如果我特征用完了，在当前子树下还是有不同类别咋办，那只能选择数量最多的了作为分类。  
好了，手工推导已经结束，接下来让代码来实现。  
首先我们想一想，需要哪些功能模块：计算信息熵的、根据特征划分、选择最佳特征划分，这三个是刚需，必须有啊，其次是打印树的、使用树来分类的，这两个的需求也很大。不过在此之前，先把数据格式定下来，这样才方便。

In [4]:
import numpy as np
import pandas as pd

In [34]:
data=pd.DataFrame({
    'No Surfacing':[1,1,1,0,0],
    'Flippers':[1,1,0,1,1],
    'Fish':[1,1,0,0,0]
})
data

,Fish,Flippers,No Surfacing
0,1,1,1
1,1,1,1
2,0,0,1
3,0,1,0
4,0,1,0


## 信息熵计算模块
思路：把数据放入后要统计分类的比例，根据比例计算信息熵并返回。

In [36]:
def calcShannonEnt(data,column):
    labels=data[column].value_counts()#统计有几类，没类多少个
    numEntries=np.sum(labels)#计算总数
    shannonEnt = 0
    for label in labels:
        prob = label/numEntries#计算当前类的概率
        shannonEnt -= prob * np.log2(prob)
    return shannonEnt

In [37]:
calcShannonEnt(data,'Fish')#测试正确

0.97095059445466858

## 根据特征划分模块

In [50]:
def splitData(data,column):
    labels=set(data[column])#该特征一共有几种分类
    retdata=[]
    for index,label in enumerate(labels):
        retdata.append(data[data[column]==label])
        del retdata[index][column]
    return retdata

In [55]:
sp=splitData(data,'No Surfacing')#测试正确
sp[1]

,Fish,Flippers
0,1,1
1,1,1
2,0,0
